In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.2/212.2 KB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.9/132.9 KB 16.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 KB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 17.2 MB/s eta 0:00:00


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim

import torchtext, datasets, math
from tqdm import tqdm

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

#make our work comparable if restarted the kernel
SEED = 1234
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

cuda:0


In [4]:
dataset = datasets.load_dataset('wikitext', 'wikitext-2-raw-v1')

Generating test split:   0%|          | 0/4358 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/36718 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3760 [00:00<?, ? examples/s]

Dataset wikitext downloaded and prepared to /root/.cache/huggingface/datasets/wikitext/wikitext-2-raw-v1/1.0.0/a241db52902eaf2c6aa732210bead40c090019a499ceb13bcbfa3f8ab646a126. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

#PreProcessing
#Tokenizing

In [5]:
tokenizer = torchtext.data.utils.get_tokenizer('spacy', language='en_core_web_sm')

tokenize_data = lambda example, tokenizer: {'tokens': tokenizer(example['text'])}  


tokenized_dataset = dataset.map(tokenize_data, remove_columns=['text'], fn_kwargs={'tokenizer': tokenizer})
print(tokenized_dataset['train'][333]['tokens'])

Map:   0%|          | 0/4358 [00:00<?, ? examples/s]

Map:   0%|          | 0/36718 [00:00<?, ? examples/s]

Map:   0%|          | 0/3760 [00:00<?, ? examples/s]

[' ', 'During', 'the', 'same', 'time', 'frame', 'as', 'the', 'Hitchcock', 'rumors', ',', 'goaltender', 'Curtis', 'Sanford', 'returned', 'from', 'his', 'groin', 'injury', 'on', 'November', '13', '.', 'He', 'made', 'his', 'first', 'start', 'of', 'the', 'season', 'against', 'the', 'Boston', 'Bruins', ',', 'losing', '2', '–', '1', 'in', 'a', 'shootout', '.', 'Sanford', 'continued', 'his', 'strong', 'play', ',', 'posting', 'a', '3', '–', '1', '–', '2', 'record', ',', '1', '@.@', '38', 'goals', 'against', 'average', 'and', '.947', 'save', 'percentage', 'over', 'his', 'next', 'six', 'games', '.', 'Sanford', 'started', '12', 'consecutive', 'games', 'before', 'Steve', 'Mason', 'made', 'his', 'next', 'start', '.', 'The', 'number', 'of', 'starts', 'might', 'not', 'have', 'been', 'as', 'numerous', ',', 'but', 'prior', 'to', 'the', 'November', '23', 'game', ',', 'Mason', 'was', 'hit', 'in', 'the', 'head', 'by', 'a', 'shot', 'from', 'Rick', 'Nash', 'during', 'pre', '@-@', 'game', 'warm', '@-@', 'ups

In [7]:
print(tokenized_dataset["train"][22]['tokens'])

[' ', 'The', 'majority', 'of', 'material', 'created', 'for', 'previous', 'games', ',', 'such', 'as', 'the', 'BLiTZ', 'system', 'and', 'the', 'design', 'of', 'maps', ',', 'was', 'carried', 'over', '.', 'Alongside', 'this', ',', 'improvements', 'were', 'made', 'to', 'the', 'game', "'s", 'graphics', 'and', 'some', 'elements', 'were', 'expanded', ',', 'such', 'as', 'map', 'layouts', ',', 'mission', 'structure', ',', 'and', 'the', 'number', 'of', 'playable', 'units', 'per', 'mission', '.', 'A', 'part', 'of', 'this', 'upgrade', 'involved', 'creating', 'unique', 'polygon', 'models', 'for', 'each', 'character', "'s", 'body', '.', 'In', 'order', 'to', 'achieve', 'this', ',', 'the', 'cooperative', 'elements', 'incorporated', 'into', 'the', 'second', 'game', 'were', 'removed', ',', 'as', 'they', 'took', 'up', 'a', 'large', 'portion', 'of', 'memory', 'space', 'needed', 'for', 'the', 'improvements', '.', 'They', 'also', 'adjusted', 'the', 'difficulty', 'settings', 'and', 'ease', 'of', 'play', 'so',

In [ ]:
tokenized_dataset

DatasetDict({
    test: Dataset({
        features: ['tokens'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['tokens'],
        num_rows: 36718
    })
    validation: Dataset({
        features: ['tokens'],
        num_rows: 3760
    })
})

#Numericalizing

In [8]:
## numericalizing
vocab = torchtext.vocab.build_vocab_from_iterator(tokenized_dataset['train']['tokens'], min_freq=3,specials=['<unk>', '<pad>', '<sos>', '<eos>']) 
vocab.set_default_index(vocab['<unk>'])   
print(len(vocab))                         
print(vocab.get_itos()[:10]) 

33258
['<unk>', '<pad>', '<sos>', '<eos>', 'the', ',', '.', 'of', 'and', 'in']


#3. Prepare the batch loader

In [9]:
unk_idx, pad_idx, sos_idx, eos_idx = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<sos>', '<eos>']

In [10]:
def get_data(dataset, vocab, batch_size):
    data = []                                                   
    for example in dataset:
        if example['tokens']:         
            #appends eos so we know it ends....so model learn how to end...                             
            tokens = example['tokens'].append('<eos>')   
            #numericalize          
            tokens = [vocab[token] for token in example['tokens']] 
            data.extend(tokens)                                    
    data = torch.LongTensor(data)                                 
    num_batches = data.shape[0] // batch_size 
    data = data[:num_batches * batch_size]                       
    data = data.view(batch_size, num_batches)          
    return data

In [11]:
batch_size = 128
train_data = get_data(tokenized_dataset['train'], vocab, batch_size)
valid_data = get_data(tokenized_dataset['validation'], vocab, batch_size)
test_data  = get_data(tokenized_dataset['test'], vocab, batch_size)

In [12]:
train_loader_length = len(list(iter(train_data)))
val_loader_length   = len(list(iter(valid_data)))
test_loader_length  = len(list(iter(test_data)))

In [13]:
class MultiHeadAttentionLayer(nn.Module):  
    def __init__(self, hid_dim, n_heads, dropout, device):
        super().__init__()
        self.hid_dim = hid_dim
        self.n_heads = n_heads
        self.head_dim = hid_dim // n_heads  #make sure it's divisible....
        
        self.fc_q = nn.Linear(hid_dim, hid_dim)
        self.fc_k = nn.Linear(hid_dim, hid_dim)
        self.fc_v = nn.Linear(hid_dim, hid_dim)
        self.fc   = nn.Linear(hid_dim, hid_dim)
        
        self.dropout = nn.Dropout(dropout)
        self.scale   = torch.sqrt(torch.FloatTensor([self.head_dim])).to(device)
        
    def forward(self, q, k, v, mask = None):
        b = q.shape[0]
        
        Q = self.fc_q(q)
        K = self.fc_k(k)
        V = self.fc_v(v)
        #Q, K, V = [b, l, h]
        
        #reshape them into head_dim
        #reshape them to [b, n_heads, l, head_dim]
        Q = Q.view(b, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        K = K.view(b, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        V = V.view(b, -1, self.n_heads, self.head_dim).permute(0, 2, 1, 3)
        #Q, K, V = [b, n_heads, l, head_dim]
        
        #e = QK/sqrt(dk)
        e = torch.matmul(Q, K.permute(0, 1, 3, 2)) / self.scale
        #e: [b, n_heads, ql, kl]
        
        if mask is not None:
            e = e.masked_fill(mask == 0, -1e10)
            
        a = torch.softmax(e, dim=-1)
        
        #eV
        x = torch.matmul(self.dropout(a), V)
        #x: [b, n_heads, ql, head_dim]
        
        x = x.permute(0, 2, 1, 3).contiguous()
        #x: [b, ql, n_heads, head_dim]
        
        #concat them together
        x = x.view(b, -1, self.hid_dim)
        #x: [b, ql, h]
        
        x = self.fc(x)
        #x = [b, ql, h]
        
        return x, a
        

In [14]:
class PositionwiseFeedforwardLayer(nn.Module):
    
    def __init__(self, hid_dim, pf_dim, dropout):
        super().__init__()
        self.fc1 = nn.Linear(hid_dim, pf_dim)
        self.fc2 = nn.Linear(pf_dim, hid_dim)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        return self.fc2(self.dropout(torch.relu(self.fc1(x))))

In [48]:
from queue import PriorityQueue
import operator

class Decoder(nn.Module):
    def __init__(self, output_dim, hid_dim, n_layers, n_heads, 
                 pf_dim, dropout, device,pad_idx,max_length = 100):
        super().__init__()
        
        self.device = device
        self.output_dim = output_dim
        self.tok_embedding = nn.Embedding(output_dim, hid_dim)
        self.pos_embedding = nn.Embedding(max_length, hid_dim)
        
        self.layers = nn.ModuleList([DecoderLayer(hid_dim, 
                                                  n_heads, 
                                                  pf_dim, 
                                                  dropout, 
                                                  device)
                                     for _ in range(n_layers)])
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)

        self.pad_idx = pad_idx
        
        self.scale = torch.sqrt(torch.FloatTensor([hid_dim])).to(device)

    def make_mask(self, x):
        
        #x = [batch size, len]
        
        pad_mask = (x != self.pad_idx).unsqueeze(1).unsqueeze(2)
        #pad_mask = [batch size, 1, 1, len]
        
        x_len = x.shape[1]
        
        sub_mask = torch.tril(torch.ones((x_len, x_len), device = self.device)).bool()
        #sub_mask = [len, len]
            
        mask = pad_mask & sub_mask
        #mask = [batch size, 1, len, len]
        
        return mask 

    def forward(self, trg):
        
        #trg = [batch size, trg len]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
                
        batch_size = trg.shape[0]
        trg_len = trg.shape[1]
        mask   = self.make_mask(trg)
        pos = torch.arange(0, trg_len).unsqueeze(0).repeat(batch_size, 1).to(self.device)          
        #pos = [batch size, trg len]
            
        trg = self.dropout((self.tok_embedding(trg) * self.scale) + self.pos_embedding(pos))
        #trg = [batch size, trg len, hid dim]
        
        for layer in self.layers:
            trg, attention = layer(trg, mask)
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        output = self.fc_out(trg)
        #output = [batch size, trg len, output dim]
            
        return output, attention
    def beam_decode(self, penalty_alpha = 0.9, max_length = 5, beam_size = 5):
        
        # Start with SOS Harry Potter is
        prompt = 'Harry Potter is '
        
        tokens = tokenizer(prompt)
        indices = [sos_idx] + [vocab[t] for t in tokens]

        decoder_input = torch.Tensor([indices]).long().to(device)
        #decoder_input: [batch size, len] = [1, 1]
        scores = torch.Tensor([0.]).to(device)
        #scores: [1]
        
        for i in range(max_length):
            
            # print(f"========Length: {i}")
            
            # Decoder prediction
            logits, _ = self.forward(decoder_input)
            #[beam_size, current dec len=i, vocab_size]
                        
            logits = logits[:, -1] 
            # Last sequence step: [beam_size, current dec len=i, vocab_size] => [beam_size, vocab_size]
            
            # print(f"{logits.shape=}")

            # Softmax
            # Log softmax is better, since beam search accumulates probability
            # if simply softmax, the probability can get too small and then become unstable
            log_probs = torch.log_softmax(logits, dim=1)
    
            # Add length penalty, otherwise, always very short sentence will win...
            penalty   = ((5 + (i+1)) / (5 + 1)) ** penalty_alpha #see https://arxiv.org/abs/1609.08144
            log_probs = log_probs / penalty
            
            # print(f"{decoder_input[:, -1]=}")
            
            # Update score where EOS has not been reached
            log_probs[decoder_input[:, -1]==eos_idx, :] = -2 #discouraged it to end
            log_probs[decoder_input[:, -1]==unk_idx, :] = -10 #very discouraged to spit out unk
            scores = scores.unsqueeze(1) + log_probs 
            # scores: [beam_size, vocab_size]
            # log_probs: [beam_size, vocab_size]

            # print(f"{log_probs.shape=}")
            # print(f"{scores.shape=}")
            #log_probs: torch.Size([1, 29475])
            #scores.shape=torch.Size([1, 29475])
            
            # Flatten scores from [beams, vocab_size] to [beams * vocab_size] to get top k, and reconstruct beam indices and token indices
            # Since we flatten it, we have to retrieve the actual beam indices and token_indices using floor division and remainder
            # You can try on paper; it will make sense
            scores, indices = torch.topk(scores.reshape(-1), beam_size) #scores: [beam_size]; #indices: [beam_size]
            beam_indices  = torch.divide   (indices, self.output_dim, rounding_mode='floor') # indices // vocab_size
            token_indices = torch.remainder(indices, self.output_dim)                        # indices %  vocab_size
            
            # print(f"{scores=}")
            # print(f"{indices.shape=}")
            
            # print(f"{indices=}")
            # print(f"{beam_indices=}")
            # print(f"{token_indices=}")
            
            # Build the next decoder input
            # For efficiency, the trick is to concatenate all hypotheses into one string and sent to decoder at once
            # We can later chop it ...
            next_decoder_input = []
            for beam_index, token_index in zip(beam_indices, token_indices):
                # print(f"{beam_index=}")
                prev_decoder_input = decoder_input[beam_index]
                # print(f"{prev_decoder_input=}")
                if prev_decoder_input[-1]==eos_idx:
                    token_index = eos_idx # once EOS, always EOS
                token_index = torch.LongTensor([token_index]).long().to(device)
                next_decoder_input.append(torch.cat([prev_decoder_input, token_index]))
                # print("here: " + " ".join([vocab.lookup_token(i) for i in next_decoder_input[-1]]) + "; score: " + str(scores[beam_index].item()))
            decoder_input = torch.vstack(next_decoder_input)
            
            # print(f"{decoder_input=}")
            
             # If all beams are finished, and the length is at least 5, exit
            if i > 5:
                if (decoder_input[:, -1]==eos_idx).sum() == beam_size:
                    break
                
        # convert the top scored sequence to a list of text tokens
        decoder_output, _ = max(zip(decoder_input, scores), key=lambda x: x[1])
        decoder_output = decoder_output[1:].cpu().numpy() # remove SOS
        
        return [vocab.lookup_token(i) for i in decoder_output if i != eos_idx] # remove EOS if exists

In [49]:
class DecoderLayer(nn.Module):
    def __init__(self, hid_dim, n_heads, pf_dim, dropout, device):
        super().__init__()
        
        self.self_attn_layer_norm = nn.LayerNorm(hid_dim)
        self.ff_layer_norm = nn.LayerNorm(hid_dim)
        self.self_attention = MultiHeadAttentionLayer(hid_dim, n_heads, dropout, device)
        self.positionwise_feedforward = PositionwiseFeedforwardLayer(hid_dim, pf_dim, dropout)
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, trg, trg_mask):
        
        #trg = [batch size, trg len, hid dim]
        #enc_src = [batch size, src len, hid dim]
        #trg_mask = [batch size, 1, trg len, trg len]
        #src_mask = [batch size, 1, 1, src len]
        
        #self attention
        _trg, attention = self.self_attention(trg, trg, trg, trg_mask)
        
        #dropout, residual connection and layer norm
        trg = self.self_attn_layer_norm(trg + self.dropout(_trg))
            
        #trg = [batch size, trg len, hid dim]
        
        #positionwise feedforward
        _trg = self.positionwise_feedforward(trg)
        
        #dropout, residual and layer norm
        trg = self.ff_layer_norm(trg + self.dropout(_trg))
        
        #trg = [batch size, trg len, hid dim]
        #attention = [batch size, n heads, trg len, src len]
        
        # return trg, attention
        return trg, attention

In [50]:
vocab_size = len(vocab)
hid_dim    = 256                
dec_layers = 3               
dec_heads  = 8
dec_pf_dim = 512
dec_dropout = 0.1     
lr = 1e-3   

model = Decoder(vocab_size, hid_dim, dec_layers, dec_heads, dec_pf_dim, dec_dropout, device, pad_idx).to(device)

In [51]:
optimizer = optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss(ignore_index = pad_idx) #combine softmax with cross entropy
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {num_params:,} trainable parameters')

The model has 18,668,266 trainable parameters


In [52]:
def get_batch(data, seq_len, idx):
    #this data is from get_data()
    #train_data.shape #[batch size, number of batches....]
    src    = data[:, idx:idx+seq_len]                   
    target = data[:, idx+1:idx+seq_len+1]  #target simply is ahead of src by 1            
    return src, target

In [53]:
def train(model, data, optimizer, criterion, batch_size, seq_len, clip, device):
    
    epoch_loss = 0
    model.train()
    # drop all batches that are not a multiple of seq_len
    # data #[batch size, bunch of tokens]
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]  #we need to -1 because we start at 0
    num_batches = data.shape[-1]
        
    for idx in tqdm(range(0, num_batches - 1, seq_len), desc='Training: ',leave=False):
        optimizer.zero_grad()
        
        src, target = get_batch(data, seq_len, idx) #src, target: [batch size, seq len]
        src, target = src.to(device), target.to(device)
        batch_size = src.shape[0]
        prediction, _ = model(src)               

        #need to reshape because criterion expects pred to be 2d and target to be 1d
        prediction = prediction.reshape(batch_size * seq_len, -1)  #prediction: [batch size * seq len, vocab size]  
        target = target.reshape(-1)
        loss = criterion(prediction, target)
        
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        epoch_loss += loss.item() * seq_len
    return epoch_loss / num_batches

In [54]:
def evaluate(model, data, criterion, batch_size, seq_len, device):

    epoch_loss = 0
    model.eval()
    num_batches = data.shape[-1]
    data = data[:, :num_batches - (num_batches -1) % seq_len]
    num_batches = data.shape[-1]
    
    decoded_batch_list = []

    with torch.no_grad():
        for idx in range(0, num_batches - 1, seq_len):
            src, target = get_batch(data, seq_len, idx)
            src, target = src.to(device), target.to(device)
            #target = [batch size, dec len]

            batch_size= src.shape[0]
            prediction, _ = model(src)
            #prediction = [batch size, dec len, output_dim]
            
            prediction = prediction.reshape(batch_size * seq_len, -1)
            target = target.reshape(-1)

            loss = criterion(prediction, target)
            epoch_loss += loss.item() * seq_len
            
    #decoding using beam_search as example (you don't need to put here, because beam_search is for intference)
    decoded_batch = model.beam_decode()
    print("Sample beam sentence: " + " ".join(decoded_batch))
            
    return epoch_loss / num_batches

In [55]:
def generate(prompt, max_seq_len, temperature, model, tokenizer, vocab, device, seed=None):
    if seed is not None:
        torch.manual_seed(seed)
    model.eval()
    tokens = tokenizer(prompt)
    indices = [vocab[t] for t in tokens]
    batch_size = 1
    with torch.no_grad():
        for i in range(max_seq_len):
            src = torch.LongTensor([indices]).to(device)
            prediction, _ = model(src)
            
            #prediction: [batch size, seq len, vocab size]
            #prediction[:, -1]: [batch size, vocab size] #probability of last vocab
            
            probs = torch.softmax(prediction[:, -1] / temperature, dim=-1)  
            prediction = torch.multinomial(probs, num_samples=1).item()    
            
            while prediction == vocab['<unk>']: #if it is unk, we sample again
                prediction = torch.multinomial(probs, num_samples=1).item()

            if prediction == vocab['<eos>']:    #if it is eos, we stop
                break

            indices.append(prediction) #autoregressive, thus output becomes input
            
            #####################################################################
            #I only do pure sampling....
            #you may want to compare here with top-k, top-p, and beam search here
            #####################################################################

    itos = vocab.get_itos()
    tokens = [itos[i] for i in indices]
    return tokens

In [56]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [57]:
import time
num_epochs = 10
seq_len  = 50
clip       = 1
best_valid_loss = float('inf')

save_path = f'{model.__class__.__name__}.pt'

train_losses = []
valid_losses = []
lr_scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.5, patience=0)

for epoch in range(num_epochs):
    
    start_time = time.time()

    train_loss = train(model, train_data, optimizer, criterion, 
                batch_size, seq_len, clip, device)
    valid_loss = evaluate(model, valid_data, criterion, batch_size, 
                seq_len, device)
    
    lr_scheduler.step(valid_loss)
    #for plotting
    train_losses.append(train_loss)
    valid_losses.append(valid_loss)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), save_path)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    
    #lower perplexity is better


Sample beam sentence: Harry Potter is known as well as a
Epoch: 01 | Time: 0m 53s
	Train Loss: 6.246 | Train PPL: 515.915
	 Val. Loss: 5.622 |  Val. PPL: 276.337


Sample beam sentence: Harry Potter is known as a " <unk>
Epoch: 02 | Time: 0m 52s
	Train Loss: 5.505 | Train PPL: 245.968
	 Val. Loss: 5.409 |  Val. PPL: 223.362


Sample beam sentence: Harry Potter is known as the " <unk>
Epoch: 03 | Time: 0m 53s
	Train Loss: 5.165 | Train PPL: 175.084
	 Val. Loss: 5.317 |  Val. PPL: 203.802


Sample beam sentence: Harry Potter is also known as " <unk>
Epoch: 04 | Time: 0m 53s
	Train Loss: 4.928 | Train PPL: 138.078
	 Val. Loss: 5.292 |  Val. PPL: 198.756


Sample beam sentence: Harry Potter is also known as " <unk>
Epoch: 05 | Time: 0m 53s
	Train Loss: 4.751 | Train PPL: 115.753
	 Val. Loss: 5.284 |  Val. PPL: 197.073


Sample beam sentence: Harry Potter is also known as the <unk>
Epoch: 06 | Time: 0m 53s
	Train Loss: 4.613 | Train PPL: 100.796
	 Val. Loss: 5.297 |  Val. PPL: 199.803


Sample beam sentence: Harry Potter is also known as the <unk>
Epoch: 07 | Time: 0m 53s
	Train Loss: 4.429 | Train PPL:  83.863
	 Val. Loss: 5.255 |  Val. PPL: 191.585


Sample beam sentence: Harry Potter is also known as the "
Epoch: 08 | Time: 0m 53s
	Train Loss: 4.349 | Train PPL:  77.433
	 Val. Loss: 5.264 |  Val. PPL: 193.235


Sample beam sentence: Harry Potter is a member of the Order
Epoch: 09 | Time: 0m 53s
	Train Loss: 4.256 | Train PPL:  70.504
	 Val. Loss: 5.246 |  Val. PPL: 189.867


Sample beam sentence: Harry Potter is a member of the Order
Epoch: 10 | Time: 0m 53s
	Train Loss: 4.215 | Train PPL:  67.721
	 Val. Loss: 5.255 |  Val. PPL: 191.497


In [58]:
model.load_state_dict(torch.load('Decoder.pt',  map_location=device))
test_loss = evaluate(model, test_data, criterion, batch_size, seq_len, device)
print(f'Test Perplexity: {math.exp(test_loss):.3f}')

Sample beam sentence: Harry Potter is a member of the Order
Test Perplexity: 174.382


In [61]:
prompt = 'Life is'
max_seq_len = 30
seed = 0
        #superdiverse   more diverse
temperatures = [0.5, 0.7, 0.75, 0.8, 1.0] 
#sample from this distribution higher probability will get more change
for temperature in temperatures:
    generation = generate(prompt, max_seq_len, temperature, model, tokenizer, 
                          vocab, device, seed)
    print(str(temperature)+'\n'+' '.join(generation)+'\n')

0.5
Life is the most influential @-@ century . It is a frequent @-@ seat of the church porch of the church , and the church 's oldest Mkhedruli inscriptions . The church

0.7
Life is the most distinguished of the most important world than half a million years . 


0.75
Life is the most distinguished of the most important world than half a million years . 


0.8
Life is the most distinguished of the most important royal court in c . 


1.0
Life is the fourth stage of the same year by a frequent version that attracts detective time we pull dee HBO , and we have released on November 5 . The song

